In [6]:
import os
import requests
import json
import pygal_maps_world.maps
from OTXv2 import OTXv2
from OTXv2 import IndicatorTypes
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

otx = OTXv2(os.environ.get('AlienVault_API_Key'))

aipdb = os.environ.get('AbuseIPDB_API_Key')
url = 'https://api.abuseipdb.com/api/v2/check'
headers = {
    'Accept': 'application/json',
    'Key': aipdb
}

mtime = (datetime.now() - timedelta(days=7)).isoformat()

#Get all pulses modified by AlienVault in the past week
pulses = otx.getall(modified_since=mtime)
indicators = []
print(len(pulses))

#Get indicators with indicator_type=ipv4
#Testing removing duplicate IPs and classifying them by threat type
#CNs = []
#for i in range(0, len(pulses)):
    #pulseindicator = pulses[i]["indicators"]
    #for indicator in pulseindicator:
        #if indicator["type"] == "IPv4":
            #print(pulses[i]["name"])
            #if "Malware" or "malware" in pulses[i]["name"]:
            #    print(pulses[i]["name"])
            #    print(indicator["indicator"])
            #    CNs.append(indicator["indicator"])
            #if "Password" or "password" in pulses[i]["name"]:
            #    print(pulses[i]["name"])
            #    print(indicator["indicator"])
            #    CNs.append(indicator["indicator"])
            #if "Threat Group" or "threat group" in pulses[i]["name"]:
            #    print(pulses[i]["name"])
            #    print(indicator["indicator"])
            #    CNs.append(indicator["indicator"])
            #if "Email" or "email" in pulses[i]["name"]:
            #    print(pulses[i]["name"])
            #    print(indicator["indicator"])
            #print(indicator["indicator"])
            #print(pulse[i][""])
            #querystring = {'ipAddress':indicator["indicator"],'maxAgeInDays':'90'}

30


In [5]:
#Get indicator origin country
CNs = []
for i in range(0, len(pulses)):
    print(pulses[i]["name"])
    pulseindicator = pulses[i]["indicators"]
    for indicator in pulseindicator:
        if indicator["type"] == "IPv4":
            print(indicator["indicator"])
            querystring = {'ipAddress':indicator["indicator"],'maxAgeInDays':'90'}
            response = requests.request(method='GET', url=url, headers=headers, params=querystring)
            decodedResponse = json.loads(response.text)
            if decodedResponse['data']['isPublic']:
                CNs.append(decodedResponse['data']['countryCode'])
                #print(decodedResponse['data']['countryCode'])

#Get the top 10 countries
from collections import Counter
top10 = Counter(CNs).most_common(10)
print(top10)

list_top10 = list(top10)
for i in range(0, len(top10)):
    top10[i] = (top10[i][0].lower(), top10[i][1])

# create a world map
worldmap = pygal_maps_world.maps.World()
  
# set the title of the map
worldmap.title = 'Top 10 Attacking Countries by AlienVault in the past week'
  
# adding the countries
worldmap.add('Frequency', {
        top10[0][0] : top10[0][1],
        top10[1][0] : top10[1][1],
        top10[2][0] : top10[2][1],
        top10[3][0] : top10[3][1],
        top10[4][0] : top10[4][1],
        top10[5][0] : top10[5][1],
        top10[6][0] : top10[6][1],
        top10[7][0] : top10[7][1],
        top10[8][0] : top10[8][1],
        top10[9][0] : top10[9][1]
})
  
worldmap.render_to_file('top10Alienvault.svg')

StripedFly: Perennially flying under the radar
5.255.86.125
45.9.148.36
45.9.148.132
45.9.148.21
Malvertising via Dynamic Search Ads delivers malware bonanza
Security Brief: TA571 Delivers IcedID Forked Loader
CloudKeys in the Air: Tracking Malicious Operations of Exposed IAM Keys
Mystic Stealer Revisited
188.40.116.251
5.42.92.211
171.22.28.235
194.87.31.123
GHOSTPULSE haunts victims using defense evasion bag o' tricks
78.24.180.93
195.201.198.179
A Retrospective on AvosLocker
Lazarus luring employees with trojanized coding challenges: The case of a Spanish aerospace company
Malicious ad served inside Bing's AI chatbot
Unmasking a Sophisticated Phishing Campaign That Targets Hotel Guests
Stealing More Than Towels: The New InfoStealer Campaign Hitting Hotels and Travel Agencies
A cascade of compromise: unveiling Lazarus' new campaign
Hong Kong residents targeted in malvertising campaigns for WhatsApp, Telegram
When PAM Goes Rogue: Malware Uses Authentication Modules for Mischief
Hackti